In [1]:
import numpy as np
import cv2
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

# 1. Read the images and cropped images and generate the excess green. 
# 2. Calculate the areas of cropped and whole images

In [4]:
# for calculating the canopy cover of GT images replace 'crops' by 'crops_gt' and 'canopy.csv' to 'canopy_gt.csv'

path = glob.glob("test_img/*.jpg")
filenames = os.listdir('test_img')

path1 = 'test_img' 
files = [os.path.splitext(filename)[0] for filename in os.listdir(path1)]

thresh = 127
cv_img = []
total_area = []

for img in path:
    
    I = cv2.imread(img)
    # get channels
    B, G, R = cv2.split(I)
    
    # calculate exgreen
    exgr = abs(2.8 * G - R - B)
    
    #convert to binary image
    thresh = 127
    im_bw = cv2.threshold(exgr, thresh, 255, cv2.THRESH_BINARY)[1]
    
    # extract fine maize part of the image
    im_bw[im_bw == 255] = 1.0
    m1 = np.multiply(I[:,:,1],im_bw)
    m2 = np.multiply(I[:,:,2],im_bw)
    m3 = np.multiply(I[:,:,0],im_bw)

    m = np.stack((m1,m2,m3),axis=2)
    
    #calculate the area as no. of non zero pixels
    total_area.append(np.count_nonzero(I))

df = []
for i in range(len(filenames)):
    
    path2 = glob.glob(os.path.join('crops',files[i], '*.jpg'))
    
    for im in path2:
        
        J = cv2.imread(im)
        B, G, R = cv2.split(J)

        # calculate exgreen
        exgr = abs(2.8 * G - R - B)
        
        #convert to binary image
        thresh = 127
        im_bw = cv2.threshold(exgr, thresh, 255, cv2.THRESH_BINARY)[1]
        
        # extract fine maize part of the cropped image
        im_bw[im_bw == 255] = 1.0
        
        m1 = np.multiply(J[:,:,1],im_bw)
        m2 = np.multiply(J[:,:,2],im_bw)
        m3 = np.multiply(J[:,:,0],im_bw)

        m = np.stack((m1,m2,m3),axis=2)
        
        #calculate the area as no. of non zero pixels
        area = np.count_nonzero(m)
        
        df.append((filenames[i],area,total_area[i]))
        
p = pd.DataFrame(df, columns =('filename','local_area','global_area'))

# Canopy Cover calculation

In [ ]:
canopy = []
final_canopy = []

#calculate canopy cover

for i in range(len(filenames)):
    
    s1 = p[p['filename'] == filenames[i]]
    
    a1 = (abs(s1['local_area'])) / s1['global_area']

    val = np.sum(a1)
    
    canopy.append((filenames[i], val))
    
q = pd.DataFrame(canopy, columns=('filename', 'canopy_cover_percent'))
q.to_csv('canopy.csv', index= False)